In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
import os
# import dotenv

# dotenv.load_dotenv()  # Uncomment if using a .env file
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"


In [6]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

# Enable tracing for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = "True"

# Load the document
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()


In [4]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
chain = load_summarize_chain(llm, chain_type="stuff")


In [7]:
result = chain.invoke(docs)
print(result["output_text"])


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_ope*******_key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [9]:

import os
from dotenv import load_dotenv

# Load environment variables from .env file if you have one
load_dotenv()

# Set the OpenAI API key and endpoint for Azure OpenAI Service
os.environ["OPENAI_API_KEY"] = "9cb47c3891a34c2e8c1ec63d8965bf2e"
os.environ["OPENAI_API_ENDPOINT"] = "https://ptsg-5cvm-oai01.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI

# Enable tracing for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = "True"

# Load the document
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

# Initialize Azure LLM
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
    openai_api_type="azure",
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

chain = load_summarize_chain(llm, chain_type="stuff")

result = chain.invoke(docs)
print(result["output_text"])


The article discusses the concept of building autonomous agents using large language models (LLMs) as their core controller. It explores the components of an LLM-powered agent system, including planning, memory, and tool use. The article also presents case studies and proof-of-concept examples of LLM-powered agents. It highlights challenges such as finite context length, reliability of natural language interface, and long-term planning. The article provides citations and references for further reading.


In [11]:
#from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain, StuffDocumentsChain



# Enable tracing for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = "True"

# Load the document
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

# Define prompt
prompt_template = """Write a 500 word summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
    openai_api_type="azure",
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

# Invoke the chain and print the result
result = stuff_chain.invoke(docs)
print(result["output_text"])


/Users/joeblack/Documents/GitHub/InternShipProject/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


The article discusses the concept of building autonomous agents using large language models (LLMs) as their core controllers. It explores various components of LLM-powered autonomous agent systems, including planning, memory, and tool use. 

In terms of planning, the article discusses task decomposition techniques such as Chain of Thought (CoT) and Tree of Thoughts (ToT), which break down complex tasks into smaller steps. It also explores the use of external classical planners for long-horizon planning.

The article then delves into the concept of memory in LLM-powered agents, including sensory memory, short-term memory, and long-term memory. It highlights the use of external vector stores and fast retrieval techniques for accessing information beyond the model's finite attention span.

The article also explores the integration of external tools in LLM-powered agents. It discusses frameworks such as MRKL and TALM, which allow agents to call external APIs for additional information and 

In [13]:
import os
import streamlit as st
from dotenv import load_dotenv
import fitz  # PyMuPDF
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain, StuffDocumentsChain

# Load environment variables from .env file if you have one
load_dotenv()

# Set the OpenAI API key and endpoint for Azure OpenAI Service
os.environ["OPENAI_API_KEY"] = "9cb47c3891a34c2e8c1ec63d8965bf2e"
os.environ["OPENAI_API_ENDPOINT"] = "https://ptsg-5cvm-oai01.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

# Enable tracing for LangChain
os.environ["LANGCHAIN_TRACING_V2"] = "True"

st.title("PDF Summarizer with Azure OpenAI")

# File uploader widget
uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")

if uploaded_file is not None:
    # Read the PDF file
    doc = fitz.open(stream=uploaded_file.read(), filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text()

    # Define prompt
    prompt_template = """Write a 500 word summary of the following:
    "{text}"
    CONCISE SUMMARY:"""
    prompt = PromptTemplate.from_template(prompt_template)

    # Define LLM
    llm = AzureChatOpenAI(
        deployment_name="gpt-35-turbo-16k",
        azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
        openai_api_type="azure",
        openai_api_version=os.getenv("OPENAI_API_VERSION"),
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0
    )

    # Define LLM chain
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # Define StuffDocumentsChain
    stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

    # Invoke the chain and print the result
    docs = [{"text": text}]
    result = stuff_chain.invoke(docs)
    st.write(result["output_text"])


2024-06-25 16:02:14.891 
  command:

    streamlit run /Users/joeblack/Documents/GitHub/InternShipProject/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
